<a href="https://colab.research.google.com/github/filsto/DigitRecognizer/blob/main/Digit3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Conv2D, Flatten, MaxPool2D, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split 

In [2]:
# Partie A - les données

from google.colab import drive
drive.mount('/content/drive')
from google.colab import files

Mounted at /content/drive


In [3]:
#loader les datasets
train_set = pd.read_csv('/content/drive/MyDrive/DATA/Digit_recognizer/train.csv', sep=",")
test_set = pd.read_csv('/content/drive/MyDrive/DATA/Digit_recognizer/test.csv', sep=",")

In [4]:
#préparer les datas
y_train=(train_set['label'])/255
X_train=(train_set.drop(['label'], axis = 1))/255

print('X_train:',X_train.shape)
print('y_train:',y_train.shape)

#redimensionner X
X_train = X_train.values.reshape(-1,28,28,1)
X_test = test_set.values.reshape(-1,28,28,1)

#categoriser y
y_train=pd.get_dummies(y_train)

#splitter
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, random_state=0)

X_train: (42000, 784)
y_train: (42000,)


In [5]:
print('X_train:',X_train.shape)
print('y_train:',y_train.shape)
print('X_valid:',X_valid.shape)
print('y_valid:',y_valid.shape)

X_train: (31500, 28, 28, 1)
y_train: (31500, 10)
X_valid: (10500, 28, 28, 1)
y_valid: (10500, 10)


In [ ]:
# Partie B - le réseau de neurones

modele = keras.Sequential([
                  layers.Conv2D(32, kernel_size=(3,3), padding='same', activation='relu', input_shape=(28,28,1)),
                  layers.MaxPool2D(),

                  layers.Conv2D(16, kernel_size=(3,3), padding='same', activation='relu'),
                  layers.MaxPool2D(),

                  layers.Flatten(),

                  layers.Dense(100, activation='relu'),
                  layers.Dropout(0.1),
                  layers.Dense(100, activation='relu'),
                  layers.Dropout(0.1),
                  layers.Dense(100, activation='relu'),
                  layers.Dropout(0.1),
                  layers.Dense(100, activation='relu'), 
                  layers.Dropout(0.1),      
                  layers.Dense(100, activation='relu'),
                  layers.Dense(10, activation='softmax')       
])

print(modele.summary())

In [7]:
modele.compile(
    optimizer = 'Adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
# Partie C - entraînement

history = modele.fit(X_train, y_train, epochs=50, validation_data=(X_valid, y_valid))


In [ ]:
history_data = pd.DataFrame(history.history)
history_data.plot(figsize=(8,5))
plt.grid=True
plt.gca().set_ylim(0,1) #regler la plage verticale sur [0-1]

In [ ]:
#history_data['loss','val_loss'].plot()
#history_data[5:,['loss','val_loss']].plot()
#history_data['accuracy','val_accuracy'].plot()

print(('best validation loss: {:0.4f}'+\
    '\nbest validation accuracy: {:0.4f}')\
    .format(history_data['val_loss'].min(),
            history_data['val_accuracy'].max()))
history_data

In [16]:
# Partie D - prédiction
y_test = modele.predict(X_test)

In [22]:
y_test = [np.argmax(x) for x in y_test]

In [24]:
y_test_df = pd.DataFrame(data = {
    'ImageId' : list(range(1, len(y_test)+1)),
    'Label' : y_test
})

In [25]:
y_test_df

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


In [28]:
# Partie E - sauvegarde

y_test_df.to_csv('/content/drive/MyDrive/DATA/Digit_recognizer/submission.csv', index=False, sep=",")